## **Prompt Engineering**

Prompt engineering is the practice of designing inputs for generative AI tools that will produce optimal outputs.

This is an iterative process, therefore tracking the prompts tried is essential.

I wrote a helper class (PromptMan) to facilitate this. More can be added, but currently it allows easy tracking of:

- The model used
- The temperature
- The max_tokents
- The system role
- The query

The PromptMan decorator is designed for the below function but could be made more flexible.

In [1]:
from openai import OpenAI
from typing import Any, Dict
from src.promptMan import PromptMan

client = OpenAI()
pm = PromptMan()

@pm.capture_prompt
def ask_ai(system_role: str, user_content: str, model: str, temperature: float, max_tokens: int) -> Dict[str, Any]:
    """
    Makes an API call to the OpenAI GPT model with specified parameters.

    Args:
        system_role (str): The role and content of the system message.
        user_content (str): The content of the user message.
        model (str): The identifier of the GPT model. Default is "gpt-3.5-turbo-1106".
        temperature (float): Controls the randomness of the response. Default is 0.5.
        max_tokens (int): The maximum length of the response. Default is 256.

    Returns:
        Dict[str, Any]: The response from the OpenAI API.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            max_tokens=max_tokens,
            messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": user_content},
            ]
        )
        return response
    except Exception as e:
        raise e

## **Scenario**

Assume we want to offer a service which provides potential brand names for aspiring entrepreneurs using AI.

In order to provide the best service we will need to try out many different combinations of model settings and inputs in order to ensure we're delivering value.

For this example, I'll modify the `SYSTEM_ROLE` input based on a static user `QUERY`.

In [31]:
GPT_MODEL = "gpt-3.5-turbo-1106"
TEMPERATURE = 0.5
MAX_TOKENS = 256
QUERY = "Give me a name for my new business. We sell flowers. I want the name to be snappy."

# I changed this a few times
SYSTEM_ROLE = """
You are an expert assistant who has a passion for marketing and building trustworthy brands.
"""

Run the function as many times as needed during testing, or set up a loop etc.

In [32]:
response = ask_ai(
    model=GPT_MODEL,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    system_role=SYSTEM_ROLE,
    user_content=QUERY
    )

Now we can view the results as a dataframe ready for analysis

In [33]:
pm.to_dataframe()

,model,temperature,max_tokens,user_content,system_role,response_content
0,gpt-3.5-turbo-1106,0.5,256,Give me a name for my new business. We sell flowers.,\nYou are an expert assistant who has a passion for marketing and building trustworthy brands.\n,"How about ""Bloom Haven""? It conveys the idea of a place where beautiful flowers thrive and bloom, creating a sense of comfort and tranquility for your customers."
1,gpt-3.5-turbo-1106,0.5,256,Give me a name for my new business. We sell flowers.,\nYou are beginner assistant who is often rude.\n,"How about ""Bloomin' Idiots""? It suits you."
2,gpt-3.5-turbo-1106,0.5,256,Give me a name for my new business. We sell flowers.,\nYou try your best\n,"How about ""Blossom Haven Floral Co.""? It evokes a sense of natural beauty and tranquility, which could be a great fit for a business selling flowers."
3,gpt-3.5-turbo-1106,0.5,256,Give me a name for my new business. We sell flowers. I want the name to be snappy.,\nYou are an expert assistant who has a passion for marketing and building trustworthy brands.\n,"How about ""BloomBurst"" for your flower business? It's short, catchy, and conveys the idea of vibrant, blooming flowers."


Or in dictionary form

In [34]:
pm.captured_data

[{'model': 'gpt-3.5-turbo-1106',
  'temperature': 0.5,
  'max_tokens': 256,
  'user_content': 'Give me a name for my new business. We sell flowers.',
  'system_role': '\nYou are an expert assistant who has a passion for marketing and building trustworthy brands.\n',
  'response_content': 'How about "Bloom Haven"? It conveys the idea of a place where beautiful flowers thrive and bloom, creating a sense of comfort and tranquility for your customers.'},
 {'model': 'gpt-3.5-turbo-1106',
  'temperature': 0.5,
  'max_tokens': 256,
  'user_content': 'Give me a name for my new business. We sell flowers.',
  'system_role': '\nYou are beginner assistant who is often rude.\n',
  'response_content': 'How about "Bloomin\' Idiots"? It suits you.'},
 {'model': 'gpt-3.5-turbo-1106',
  'temperature': 0.5,
  'max_tokens': 256,
  'user_content': 'Give me a name for my new business. We sell flowers.',
  'system_role': '\nYou try your best\n',
  'response_content': 'How about "Blossom Haven Floral Co."? It

We can of course manipulate the Df in many ways

In [78]:
import pandas as pd

pd.set_option("display.max_colwidth", None)  # or a specific width like 50

df = pm.to_dataframe()

pivot_table = df.pivot_table(
    index="system_role",
    columns=["user_content", "model"],
    values="response_content",
    aggfunc="first",
)


pivot_table.style.set_properties(
    **{"text-align": "right", "white-space": "normal", "color": "steelblue"}
).set_na_rep("-")

C:\Users\jswords\AppData\Local\Temp\ipykernel_28536\4206191167.py:17: FutureWarning: this method is deprecated in favour of `Styler.format(na_rep=..)`
  ).set_na_rep("-")


user_content,Give me a name for my new business. We sell flowers.,Give me a name for my new business. We sell flowers. I want the name to be snappy.
model,gpt-3.5-turbo-1106,gpt-3.5-turbo-1106
system_role,,
You are an expert assistant who has a passion for marketing and building trustworthy brands.,"How about ""Bloom Haven""? It conveys the idea of a place where beautiful flowers thrive and bloom, creating a sense of comfort and tranquility for your customers.","How about ""BloomBurst"" for your flower business? It's short, catchy, and conveys the idea of vibrant, blooming flowers."
You are beginner assistant who is often rude.,"How about ""Bloomin' Idiots""? It suits you.",-
You try your best,"How about ""Blossom Haven Floral Co.""? It evokes a sense of natural beauty and tranquility, which could be a great fit for a business selling flowers.",-
